In [ ]:
import requests
import json
import os

In [ ]:
from quantz_repo import utils
cfg = utils.initialize_quantz_config()
print(cfg.fred_api_key)
print(cfg.finnhub_api_key)
print(cfg.iexcloud_api_token)
print(cfg.quandl_api_key)
#print(cfg.whatever)

In [ ]:
class Fred:
    '''
    直接返回获取的 json 或者 抛出异常提示错误
    '''
    def __init__(self):
        '''
        get Fred 数据
        TODO: 通过配置或环境变量获取 Api Key

        '''
        self.api_key = cfg.fred_api_key
        #get_fred_api_key_from_cfg()
        if self.api_key is None:
            raise Exception('Fred Api Key can not be empty')
        self.path='https://api.stlouisfed.org/fred'

    def __get(self, args):
        print('__get:%s' % self.path)
        print(args)
        resp = None
        if len(args) == 0:
            resp = requests.get(self.path, params={'api_key':self.api_key, 'file_type':'json'})
        elif len(args) == 1:
            params={'api_key':self.api_key, 'file_type':'json'}
            params.update(args[0])
            print(params)
            resp = requests.get(self.path, params=params)
        else:
            print('Invalid args%s' % args)
            return None
        # TODO: 增加网络错误的处理
        # print(resp.json())
        return resp.json()

    def __getattr__(self, name):
        '''
        __getattr__ 在需要的属性没有找到的情况下会调用
        __getattribute__ 在读取任何属性的时候都会调用
        '''
        self.path = '%s/%s' % (self.path, name)
        return self

    def __str__(self):
        return self.path

    def __call__(self,*args):
        return self.__get(args)

In [ ]:
wei_json = Fred().series.observations({'series_id':'wei', 'sort_order':'desc'})


In [ ]:
print(len(wei_json['observations']))
print(wei_json)

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
# wei_df = pd.read_json(StringIO(str(wei_json['observations'])))


In [ ]:
wei_df = DataFrame(wei_json['observations'])
wei_df['value'] = wei_df['value'].astype(np.float)
wei_df['date'] = wei_df['date'].astype(np.datetime64) 
wei_df['date'] = wei_df['date'].astype(np.int64)
wei_df['date'] = wei_df['date']/100000
wei_df['date'] = wei_df['date'].astype(np.int64)
wei_df.drop(axis=1, inplace=True, columns=['realtime_start', 'realtime_end'])
print(wei_df.dtypes)
print(wei_df.head)
